# Week 12: Advanced Text Analysis Topics & Machine Learning 2

# 1. Word2Vec으로 문장을 벡터로 변환
* 사용된 샘플코드는 <파이썬을 이용한 머신러닝, 딥러닝 실전 개발 입문> p.279를 참고하였음.


* Word2Vec 은 문장 내부의 단어를 벡터로 변환하는 도구

* 단어의 연결구조를 기반으로 단어의 연관성을 벡터로 표현함 → 단어의 의미를 파악하는데 도움이 됨

* Co-occurrence 연결망 (one-hot encoding) → 도큐멘트 내에서 연결된 단어를 파악하는데 도움 (local structure)

    * 그러나 유사한 단어, 즉 “강아지”와 “댕댕이”는 서로 다른 벡터로 인식
    * 단어 간의 유사성을 파악하지 못함
    
* 단어를 벡터로 구조화 하면 단어들 사이의 연관성을 파악할 수 있음
* 단어를 벡터화 하면 벡터의 연산이 가능 

    * 아빠-남자+여자 = 엄마
    * 왕자-남성+여성 = 공주

* [http://w.elnn.kr/search/](http://w.elnn.kr/search/) 참고


## Word2Vec 모델 만들기

In [ ]:
# import libraries
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from gensim.models import word2vec

In [ ]:
# utf-16 인코딩으로 파일 열고 텍스트만 선택한 후 text 변수에 저장
#files = ["BEXX0003", "2BEXXX06", "2BEXXX05", "2BEXXX03", "2BEXXX01"] ## 토지 5권. 수업에서는 한권만 training 해보자.
files = ["BEXX0003"]
text = ""
for fname in files:
    fp = codecs.open("data/" + fname + ".txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("body")
    text += body.getText()

In [ ]:
# 텍스트를 한줄 씩 형태소 처리 (Twitter 형태소 분석기를 사용)
twitter = Okt()
results = []
lines = text.split("\n")
for line in lines:
    # 형태소 분석
    # 단어의 기본형 사용
    malist = twitter.pos(line, norm=True, stem=True)
    r = []
    for word in malist:
        # print(word)
        # 조사, 어미, 구두점은 제외
        if not word[1] in ["Josa", "Eomi", "Puncuation"]:
            r.append(word[0])
    rl = (" ".join(r)).strip()
    results.append(rl)
#     print(rl)

In [ ]:
# 형태소 분석한 내용을 파일로 출력
wakati_file = 'toji.wakati'
with open(wakati_file, "w", encoding='utf-8') as fp:
    fp.write("\n".join(results))

In [ ]:
# Word2Vec 모델 만들기
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data, size=200, window=10, hs=1, min_count=2, sg=1)
model.save("toji.model")
print("OK")

## Word2Vec 모델 탐색

In [ ]:
# 저장한 모델 불러오기 
from gensim.models import word2vec
model = word2vec.Word2Vec.load("toji.model")

In [ ]:
model.wv.most_similar(positive=["술"])

In [ ]:
model.wv.most_similar(positive=["남자"])

In [ ]:
model.wv.most_similar(positive=["주모", "남자"], negative=["여자"])

# 2. LDA (Latent Dirichlet Allocation)
* http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/


<img src="image/lda.png" alt="drawing" width="800"/>


* LDA는 특정 토픽에 특정 단어가 나타날 확률
* 노란색 토픽엔 gene이라는 단어가 등장할 확률이 0.04, dna는 0.02, genetic은 0.01 -> 유전자 관련 주제임을 알 수 있음
* 문서 자체를 보면 빨강, 파랑 토픽 보다는 노란색의 비중이 많음 -> 따라서 이 문서의 메인 주제는 역시 유전자일 가능성이 높음

In [ ]:
# import libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from gensim import corpora, models

In [ ]:
# Text preprocessing
# lemmatizing and remove stopwords
def lemmatize_stemming(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
# create sample documents
doc_a = "I like to eat broccoli and bananas."
doc_b = "I ate a banana and spinach smoothie for breakfast."
doc_c = "Chinchillas and kittens are cute."
doc_d = "My sister adopted a kitten yesterday."
doc_e = "Look at this cute hamster munching on a piece of broccoli."
 
# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
 
# list for tokenized documents in loop
texts = []
 
# loop through document list
for i in doc_set:

    stemmed_tokens = preprocess(i)
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
 
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
 
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

In [ ]:
# print topics
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
test_document = 'Look at this cute hamster munching on a piece of broccoli.'
bow_vector = dictionary.doc2bow(preprocess(test_document))
for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamodel.print_topic(index, 5)))

In [ ]:
unseen_document = 'I used to eat eggs for breakfast'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamodel.print_topic(index, 5)))

# 3. Sentiment Analysis using Google Translator

In [ ]:
# import libraries
import glob
import nltk
from nltk.classify import NaiveBayesClassifier
from googleapiclient.discovery import build

In [ ]:
# setup review polarity data
# http://www.cs.cornell.edu/people/pabo/movie-review-data

def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

pos = []
pos_path = 'review_polarity/txt_sentoken/pos/*.txt'
pos_files = glob.glob(pos_path)
for file in pos_files:
    with open(file) as f:
        for i in f:
            pos.append([format_sentence(i), 'pos'])

neg = []
neg_path = 'review_polarity/txt_sentoken/neg/*.txt'
neg_files = glob.glob(neg_path)
for file in neg_files:
    with open(file) as f:
        for i in f:
            neg.append([format_sentence(i), 'neg'])

training = pos[:int((.8)*len(pos))] + neg[:int((.8)*len(neg))]
test = pos[int((.8)*len(pos)):] + neg[int((.8)*len(neg)):]

classifier = NaiveBayesClassifier.train(training)
classifier.show_most_informative_features()

In [ ]:
example1 = "Python is an awesome language!"
print("---\n" + example1)
print(classifier.classify(format_sentence(example1)))

In [ ]:
example2 = "I don't think I can live without Python."
print("---\n" + example2)
print(classifier.classify(format_sentence(example2)))

In [ ]:
example3 = "I hate pineapple pizza."
print("---\n" + example3)
print(classifier.classify(format_sentence(example3)))

In [ ]:
# Sentiment Analysis using Google Translator

text = ["올해 영화관에서 본 영화중 최고였다매혹적이고 아름답고 슬프기도하다", 
        "진지하고 여운이 남는 영화에요 추천합니다", 
        "재미 드럽게 없음..포와로역 외모도 안어울리고, 지루하고, 긴장감도 없고... 실망입니다. 졸려요..", 
        "사람 낚는 올해 최고의 낚시 작품!1점 주는 것도 아까운 최고의 서스펜스 네이버 평점 알바 놈들을 반드시 잡고싶게 만드는 초특급 추리영화!! 꼭 보시라~ 나만 보고 당하면 열받잖아?이감독 이게 몇번째야... 내돈 내놔 이시끼야!", 
        "이 영화가 구린이유1.추리하지않는 탐정영화추리하는과정은 제대로 나오지도않고 추리해서 나온 결과만 말함ㅋㅋ 읭??저건어떻게 알아낸거지라는 의문이 시도때도없이 든다.2. 수면제임긴장감이 하나도 없다. 추리하는중에도..3. 다소 억지스러운 스토리", 
        "너무 차분하고 재미없다는 평을 여러개 봤지만 저는 재미있었어요....겨울 배경이 지금 우리계절과도 맞아서 보는내내 사계절이 뚜렷한 우리나라가 좋기도 했네요...추억과 향수를 불러내는 오랜만에 보는 좋은영화입니다..지나치게 잔인하지도 선정적이지도 않아요", 
        "평소에 잠을 푹 못잤는데 심야로 보다가 아주 잘 자고갑니다 너무너무고마운영화 불면증치료에 좋은 힐링물"]

## google translate
googleKey = ''
service = build('translate', 'v2', developerKey = googleKey)

results = service.translations().list(source='ko', target='en', q=text).execute()

for i in range(len(text)):
    print("---\n" + text[i])
    en_txt = results['translations'][i]['translatedText'].replace('&#39;',"'")
    print(en_txt)
    print(classifier.classify(format_sentence(en_txt)))

In [ ]:
# Sentiment Analysis using Google Translator - 영화평이 아닌 경우

text = ["엥 이게 지금 1위라고??ㅋㅋㅋ에반데",
       "노래좋다 치자 그렇다고 냉정하게 봤을때 1위할만한 곡ㅇ른 아닌데?"]

## google translate
googleKey = ''
service = build('translate', 'v2', developerKey = googleKey)

results = service.translations().list(source='ko', target='en', q=text).execute()

for i in range(len(text)):
    print("---\n" + text[i])
    en_txt = results['translations'][i]['translatedText'].replace('&#39;',"'")
    print(en_txt)
    print(classifier.classify(format_sentence(en_txt)))

# 4. Image Analysis using AWS

* https://aws.amazon.com/ko/rekognition/

In [ ]:
import boto3

# BUCKET과 KEY는 AWS의 S3 스토리지에 생성한 것을 사용 (개인마다 다름)
BUCKET = "hcidphoto" 
KEY = "beach.jpg"

def detect_labels(bucket, key, max_labels=10, min_confidence=90, region="us-west-2"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.detect_labels(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        MaxLabels=max_labels,
        MinConfidence=min_confidence,
    )
    return response['Labels']

for label in detect_labels(BUCKET, KEY):
    print("{Name} - {Confidence}%".format(**label))

In [ ]:
# '**' 표기법을 사용하면 dictionary 의 키를 인자로 하여 값을 가져올 수 있음.

table = {'Sjoerd': 4127, 'Jack': 4098, 'Dcab': 8637678}
print('Jack: {Jack:d}; Sjoerd: {Sjoerd:d}; Dcab: {Dcab:d}'.format(**table))
Jack: 4098; Sjoerd: 4127; Dcab: 8637678

In [ ]:
from pprint import pprint
import boto3

# BUCKET과 KEY는 AWS의 S3 스토리지에 생성한 것을 사용 (개인마다 다름)
BUCKET = "hcidphoto"
KEY = "person.jpg"

def detect_faces(bucket, key, attributes=['ALL'], region="us-west-2"):
    rekognition = boto3.client("rekognition", region)
    response = rekognition.detect_faces(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        Attributes=attributes,
    )
    return response['FaceDetails']

for face in detect_faces(BUCKET, KEY):
    pprint(face)